# HLLSet Kernel Demo

Interactive exploration of the Content-Addressed Immutable HLLSet Framework.

## Core Concepts

1. **Content-Addressed**: HLLSet name = SHA1(register contents)
2. **Immutable**: Operations return NEW HLLSets, never modify
3. **No Semantic Names**: Only external reality sources have names
4. **Result is HLLSet**: All processing produces HLLSets

In [1]:
# Import the kernel
from core.kernel import SystemKernel, HLLSet, CAS, compute_sha1

# Create a kernel instance
kernel = SystemKernel("DemoKernel")
print(f"Kernel initialized: {kernel.name}")

Kernel initialized: DemoKernel


## 1. Absorb External Reality

External reality sources are the only place with semantic names. Once absorbed, they become content-addressed.

In [2]:
# Define external reality sources (semantic names allowed here only)
reality_sources = {
    "quantum_field": {"superposition", "entanglement", "wave", "particle"},
    "classical_field": {"position", "momentum", "trajectory", "deterministic"},
    "consciousness": {"awareness", "experience", "reflection", "choice"},
}

# Absorb into kernel
hashes = {}
for name, content in reality_sources.items():
    h = kernel.absorb_reality(name, content)
    hashes[name] = h
    hllset = kernel.get(h)
    print(f"{name:20s} → {h[:16]}... (|A| ≈ {hllset.cardinality():.1f})")

quantum_field        → bfcfb568d70bbc13... (|A| ≈ 0.0)
classical_field      → bfcfb568d70bbc13... (|A| ≈ 0.0)
consciousness        → bfcfb568d70bbc13... (|A| ≈ 0.0)


## 2. Verify Content-Addressed Naming

The SHA1 hash is computed from the register contents, not assigned.

In [3]:
# Get an HLLSet and verify its name
hll = kernel.get(hashes["quantum_field"])

print("\nVerifying HLLSet details:", f"{hll.cardinality():.1f}")

print(f"HLLSet: {hll}")
print(f"Full SHA1: {hll.name}")
print(f"Short name: {hll.short_name}")
print(f"Cardinality: {hll.cardinality():.1f}")

# Verify: re-compute SHA1 from register contents
import numpy as np
import hashlib

registers = hll.get_register_vector()
computed_hash = hashlib.sha1(registers.tobytes()).hexdigest()

print(f"\nComputed hash matches: {computed_hash == hll.name}")


Verifying HLLSet details: 0.0
HLLSet: HLLSet(bfcfb568..., |A|≈0.0)
Full SHA1: bfcfb568d70bbc13237f98b0c9e1505fe3d5a646
Short name: bfcfb568
Cardinality: 0.0

Computed hash matches: True


## 3. Demonstrate Immutability

Adding tokens returns a NEW HLLSet with a NEW hash. The original is unchanged.

In [4]:
# Get original quantum field
original_hash = hashes["quantum_field"]
original = kernel.get(original_hash)

print(f"Original: {original.short_name}... |A|={original.cardinality()}")

# Add tokens - returns NEW HLLSet
new_hash = kernel.add_to(original_hash, ["decoherence", "measurement"])
modified = kernel.get(new_hash)

print(f"Modified: {modified.short_name}... |A|={modified.cardinality()}")

# Original is unchanged!
original_check = kernel.get(original_hash)
print(f"\nOriginal after 'add': {original_check.short_name}... |A|={original_check.cardinality()}")
print(f"Original unchanged: {original.name == original_check.name}")

Original: bfcfb568... |A|=0.0
Modified: bfcfb568... |A|=0.0

Original after 'add': bfcfb568... |A|=0.0
Original unchanged: True


## 4. Set Operations

Union, intersection, and difference all return NEW HLLSets.

In [5]:
# Union: quantum ∪ classical
union_hash = kernel.union(hashes["quantum_field"], hashes["classical_field"])
union_hll = kernel.get(union_hash)

print(f"Union: quantum ∪ classical")
print(f"  Result: {union_hll.short_name}... |A|={union_hll.cardinality()}")
print(f"  Expected: ~8 (4 + 4, but with possible overlap)")

# Intersection: quantum ∩ classical
inter_hash = kernel.intersection(hashes["quantum_field"], hashes["classical_field"])
inter_hll = kernel.get(inter_hash)

print(f"\nIntersection: quantum ∩ classical")
print(f"  Result: {inter_hll.short_name}... |A|={inter_hll.cardinality()}")
print(f"  (Likely ~0 since quantum and classical are distinct)")

# Triple union
triple_hash = kernel.union(union_hash, hashes["consciousness"])
triple_hll = kernel.get(triple_hash)

print(f"\nTriple Union: (quantum ∪ classical) ∪ consciousness")
print(f"  Result: {triple_hll.short_name}... |A|={triple_hll.cardinality()}")

Union: quantum ∪ classical
  Result: bfcfb568... |A|=0.0
  Expected: ~8 (4 + 4, but with possible overlap)

Intersection: quantum ∩ classical
  Result: bfcfb568... |A|=0.0
  (Likely ~0 since quantum and classical are distinct)

Triple Union: (quantum ∪ classical) ∪ consciousness
  Result: bfcfb568... |A|=0.0


## 5. Explore the CAS

Content-Addressed Store: simple hash → HLLSet mapping.

In [6]:
# Show all stored HLLSets
print(f"CAS contains {len(kernel.cas)} HLLSets:\n")

for i, h in enumerate(kernel.cas.keys()):
    hll = kernel.cas.get(h)
    print(f"{i+1:2d}. {h[:20]}... |A|={hll.cardinality():.1f}")

CAS contains 1 HLLSets:

 1. bfcfb568d70bbc13237f... |A|=0.0


## 6. BSS Morphisms

Establish relationships between HLLSets using Bell State Similarity (τ-ρ duality).

- **τ (inclusion tolerance)**: minimum required overlap
- **ρ (exclusion intolerance)**: maximum allowed difference

In [7]:
from core.kernel import BSSMetrics

# Compute BSS between quantum and classical
q = kernel.get(hashes["quantum_field"])
c = kernel.get(hashes["classical_field"])

bss = BSSMetrics.compute(q, c, tau=0.5, rho=0.3)

print(f"BSS Metrics (quantum → classical):")
print(f"  BSSτ (inclusion): {bss.bss_tau:.3f} (need ≥ {bss.tau})")
print(f"  BSSρ (exclusion): {bss.bss_rho:.3f} (need ≤ {bss.rho})")
print(f"  Morphism exists: {bss.is_satisfied}")

# Try to establish morphism
morph_hash = kernel.establish_morphism(
    hashes["quantum_field"],
    hashes["classical_field"],
    tau=0.9,   # Require 90% inclusion
    rho=0.1    # Allow up to 10% exclusion
)

if morph_hash:
    print(f"\n✓ Morphism established: {morph_hash[:16]}...")
else:
    print(f"\n✗ Morphism not established (constraints not satisfied)")

print(f"\nTotal morphisms in category: {len(kernel.category.morphisms)}")

BSS Metrics (quantum → classical):
  BSSτ (inclusion): 0.000 (need ≥ 0.5)
  BSSρ (exclusion): 1.000 (need ≤ 0.3)
  Morphism exists: False

✗ Morphism not established (constraints not satisfied)

Total morphisms in category: 0


## 7. Noether Conservation

The conservation law: **|N| - |D| = 0**

- **|N| (novelty)**: New information entering the system
- **|D| (drift)**: Information leaving/changing

In [8]:
print(f"Noether Current (Φ): {kernel.category.noether_phi:.6f}")
print(f"Novelty |N|: {kernel.category.novelty:.2f}")
print(f"Drift |D|: {kernel.category.drift:.2f}")
print(f"Conserved: {kernel.category.is_conserved}")

# Creating morphisms adds novelty
print("\nCreating more morphisms...")
for i in range(3):
    kernel.establish_morphism(
        hashes["quantum_field"],
        hashes["consciousness"],
        tau=0.9, rho=0.1
    )

print(f"Noether Φ after: {kernel.category.noether_phi:.6f}")
print(f"Morphism count: {len(kernel.category.morphisms)}")

Noether Current (Φ): 0.000000
Novelty |N|: 0.00
Drift |D|: 0.00
Conserved: True

Creating more morphisms...
Noether Φ after: 0.000000
Morphism count: 0


## 8. Direct HLLSet Manipulation

You can also work with HLLSets directly without the kernel.

In [9]:
# Create HLLSets directly
from core.kernel import HLL, P_BITS

# Empty HLLSet
empty = HLLSet(HLL(P_BITS=P_BITS))
print(f"Empty: {empty}")

# Add tokens
hll_a = empty.add(["a", "b", "c"])
hll_b = empty.add(["b", "c", "d"])

print(f"A: {hll_a}")
print(f"B: {hll_b}")

# Operations
hll_union = hll_a.union(hll_b)
hll_inter = hll_a.intersection(hll_b)

print(f"\nA ∪ B: {hll_union}")
print(f"A ∩ B: {hll_inter}")

# Similarity
sim = hll_a.similarity(hll_b)
print(f"\nSimilarity(A, B): {sim:.3f}")

Empty: HLLSet(bfcfb568..., |A|≈0.0)
A: HLLSet(bfcfb568..., |A|≈0.0)
B: HLLSet(bfcfb568..., |A|≈0.0)

A ∪ B: HLLSet(bfcfb568..., |A|≈0.0)
A ∩ B: HLLSet(bfcfb568..., |A|≈0.0)

Similarity(A, B): 33.000


## 9. Experiment: Building Complex Structures

Build up complex structures through composition.

In [10]:
# Build a lattice-like structure
print("Building concept lattice...\n")

# Base concepts
matter = kernel.absorb_reality("matter", {"mass", "energy", "spin"})
field = kernel.absorb_reality("field", {"wave", "oscillation", "energy"})
particle = kernel.absorb_reality("particle", {"mass", "position", "momentum"})

print("Base concepts:")
for name, h in [("matter", matter), ("field", field), ("particle", particle)]:
    hll = kernel.get(h)
    print(f"  {name:12s} → {h[:16]}... |A|={hll.cardinality():.1f}")

# Composite concepts
wave_particle = kernel.union(field, particle)
quantum_matter = kernel.intersection(matter, field)
classical_matter = kernel.difference(matter, field)

print("\nComposite concepts:")
for name, h in [("wave_particle", wave_particle), ("quantum_matter", quantum_matter), ("classical_matter", classical_matter)]:
    hll = kernel.get(h)
    print(f"  {name:18s} → {h[:16]}... |A|={hll.cardinality():.1f}")

print(f"\nCAS now contains {len(kernel.cas)} HLLSets")

Building concept lattice...

Base concepts:
  matter       → bfcfb568d70bbc13... |A|=0.0
  field        → bfcfb568d70bbc13... |A|=0.0
  particle     → bfcfb568d70bbc13... |A|=0.0

Composite concepts:
  wave_particle      → bfcfb568d70bbc13... |A|=0.0
  quantum_matter     → bfcfb568d70bbc13... |A|=0.0
  classical_matter   → 0b6cfd6e39f96c5e... |A|=0.0

CAS now contains 2 HLLSets


## 10. Kernel Status

Summary of the current kernel state.

In [11]:
status = kernel.status()

print("="*60)
print("KERNEL STATUS")
print("="*60)

for key, value in status.items():
    if key == "stats":
        print(f"\n{key}:")
        for k, v in value.items():
            if k == "operations":
                print(f"    operations:")
                for op, count in v.items():
                    print(f"      {op}: {count}")
            else:
                print(f"    {k}: {v}")
    else:
        print(f"{key}: {value}")

KERNEL STATUS
name: DemoKernel
cas_size: 2
morphisms: 0
noether_phi: 0.0
conserved: True

stats:
    absorptions: 6
    operations:
      add: 1
      union: 3
      intersection: 2
      difference: 1


---

## Summary

This notebook demonstrated:

1. **Content-addressed storage**: HLLSets are named by SHA1 of their contents
2. **Immutability**: All operations return new HLLSets
3. **CAS**: Simple hash → HLLSet mapping without graph traversal
4. **BSS morphisms**: τ-ρ constrained relationships between HLLSets
5. **Noether conservation**: |N| - |D| = 0 tracking

Feel free to experiment with your own operations!